Import Libraries

In [1]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium # map rendering library
!pip install foursquare
!pip install git+https://github.com/dacog/foursquare_api_tools.git#egg=foursquare_api_tools
import foursquare as fs
from foursquare_api_tools import foursquare_api_tools as ft
print('Folium installed')
print('Libraries imported.')

  Created wheel for foursquare: filename=foursquare-1!2019.9.11-cp36-none-any.whl size=20354 sha256=5cd255d2b3e92e806b7cf3e636e0950ac525f70cbb781f1c18fb895fe895ac8c
  Stored in directory: /root/.cache/pip/wheels/53/6c/d9/0810f42ef7521037af97032caab9411144ab0efab2aed8300f
Successfully built foursquare
  Cloning https://github.com/dacog/foursquare_api_tools.git to /tmp/pip-install-t7ikaecu/foursquare-api-tools
  Running command git clone -q https://github.com/dacog/foursquare_api_tools.git /tmp/pip-install-t7ikaecu/foursquare-api-tools
  Created wheel for foursquare-api-tools: filename=foursquare_api_tools-0.2-cp36-none-any.whl size=4298 sha256=69cf5d583256875e52cb3194f6d24f62458d6dc8d12f0625e48e1bc5490eaec0
  Stored in directory: /tmp/pip-ephem-wheel-cache-537qqtya/wheels/6f/3c/fd/6f3f03526dd57baad6e211d601d808ae3efd032480218ae227
Successfully built foursquare-api-tools
Folium installed
Libraries imported.


Define Client credentials

In [2]:
CLIENT_ID = '2WEIVPXNKBQZB1W2O220DXKIIMIMVOJQLG2NXSZZYO4KJI3L' # your Foursquare ID
CLIENT_SECRET = 'ZIZYUWH5IUNFJ3MVBIUESNHJ0BSO5MX3MPTCYF4WYGMHAWVT' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: 2WEIVPXNKBQZB1W2O220DXKIIMIMVOJQLG2NXSZZYO4KJI3L


Define the corresponding URL, send the GET Request to Foursquare API and examine the results

In [0]:
# type your answer here
LIMIT = 500 # Maximum is 100
radius = 1500
cities = ['Brunswick VIC', 'Auckland', 'Wellington', 'Christchurch', 'Hamilton NZ']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&radius={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        radius,
        "4bf58dd8d48988d1d2941735") # sushi restaurant id
    results[city] = requests.get(url).json()

In [0]:
#results

Create a dataframe called sushi_venues to contain each cities' sushi venues' name, address, latitude and longitude.

In [0]:
sushi_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    sushi_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    sushi_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

Create a Folium map for each city, with markers added for each sushi venue within the definied 2km radius.
Determine how many sushi venues are found within this radius.

In [6]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=12)

    # add markers to map
    for lat, lng, label in zip(sushi_venues[city]['Lat'], sushi_venues[city]['Lng'], sushi_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
    print(f"Total number of sushi places in {city} within {radius}m of City Centre= ", results[city]['response']['totalResults'])


Total number of sushi places in Brunswick VIC within 1500m of City Centre=  54
Total number of sushi places in Auckland within 1500m of City Centre=  46
Total number of sushi places in Wellington within 1500m of City Centre=  35
Total number of sushi places in Christchurch within 1500m of City Centre=  8
Total number of sushi places in Hamilton NZ within 1500m of City Centre=  18


Visually inspect the maps

In [7]:
maps[cities[0]]

In [8]:
maps[cities[1]]

In [9]:
maps[cities[2]]

In [10]:
maps[cities[3]]

In [11]:
maps[cities[4]]

To determine the best location to find a house/apartment which would limit the mean distance to each sushi venue, find the mean of each cities' sushi venues' latitudes and longitudes, and add this location to the map.

In [12]:
for city in cities:
  print("the mean latitude and longitude for (city) are:"([sushi_venues[city]['Lat'].mean(), sushi_venues[city]['Lng'].mean()]))

TypeError: ignored

To get an indication of restaurant density, find the mean distance to the mean location for each of the restaurants in each city.

In [0]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=12)
    venues_mean_coor = [sushi_venues[city]['Lat'].mean(), sushi_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(sushi_venues[city]['Lat'], sushi_venues[city]['Lng'], sushi_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,sushi_venues[city][['Lat','Lng']].values)))

In [0]:
for city in cities:
  print("the mean latitude and longitude for (city) are:"([sushi_venues[city]['Lat'].mean(), sushi_venues[city]['Lng'].mean()]))


In [0]:
maps[cities[0]]

In [0]:
maps[cities[1]]

In [0]:
maps[cities[2]]

In [0]:
maps[cities[3]]

In [0]:
maps[cities[4]]